## Question 1

In [36]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

im = cv.imread('the_berry_farms_sunflower_field.png',cv.IMREAD_REDUCED_COLOR_4)
#sample_g = rgb2gray(sample)
# fig, ax = plt.subplots(1,2,figsize=(10,5))
cv.imshow('image',im)
# ax[1].imshow(sample_g,cmap='gray')
# ax[0].set_title('Colored Image',fontsize=15)
# ax[1].set_title('Grayscale Image',fontsize=15)
#plt.show()

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


In [35]:
from skimage.feature import blob_dog, blob_log, blob_doh

ModuleNotFoundError: No module named 'skimage'